In [7]:
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras.optimizers import SGD
from keras.datasets import cifar10
import numpy as np
import argparse
import sys
import os

In [9]:
# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#x_train = x_train / 255.
#x_test = x_test / 255.
# apply mean subtraction to the data
mean = np.mean(x_train, axis=0)
x_train-= mean
x_test -= mean
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [10]:
# 建立 ResNet 模型
model = resnet(input_shape=(32,32,3)) 
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_32 (Conv2D)              (None, 32, 32, 16)   448         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_29 (BatchNo (None, 32, 32, 16)   64          conv2d_32[0][0]                  
__________________________________________________________________________________________________
activation_29 (Activation)      (None, 32, 32, 16)   0           batch_normalization_29[0][0]     
__________________________________________________________________________________________________
conv2d_33 

In [11]:
NUM_EPOCHS = 30
INIT_LR = 1e-1
def poly_decay(epoch):
    # initialize the maximum number of epochs, base learning rate,
    # and power of the polynomial
    maxEpochs = NUM_EPOCHS
    baseLR = INIT_LR
    power = 1.0

    # compute the new learning rate based on polynomial decay
    alpha = baseLR * (1 - (epoch / float(maxEpochs))) ** power

    # return the new learning rate
    return alpha


In [12]:
#setup callback
callbacks = [LearningRateScheduler(poly_decay)]

In [14]:
batch_size = 128 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = NUM_EPOCHS # 訓練整個資料集共 30個循環

aug = ImageDataGenerator(width_shift_range=0.1,height_shift_range=0.1, horizontal_flip=True,fill_mode="nearest")

opt = SGD(lr=INIT_LR, momentum=0.9)
#model = ResNet.build(32, 32, 3, 10, (9, 9, 9),(64, 64, 128, 256), reg=0.0005)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

model.fit_generator(
    aug.flow(x_train, y_train, batch_size=batch_size),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // 128, epochs=NUM_EPOCHS,
    callbacks=callbacks, verbose=1)

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

W0810 22:38:55.991400 4521809344 deprecation.py:323] From /Users/vincent/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/30
390/390 [==============================] - 879s 2s/step - loss: 2.1223 - acc: 0.4274 - val_loss: 2.0439 - val_acc: 0.4792
Epoch 2/30
390/390 [==============================] - 868s 2s/step - loss: 1.6278 - acc: 0.5995 - val_loss: 1.8327 - val_acc: 0.5363
Epoch 3/30
390/390 [==============================] - 928s 2s/step - loss: 1.3791 - acc: 0.6743 - val_loss: 1.5666 - val_acc: 0.6425
Epoch 4/30
390/390 [==============================] - 974s 2s/step - loss: 1.2156 - acc: 0.7207 - val_loss: 1.5426 - val_acc: 0.6048
Epoch 5/30
390/390 [==============================] - 901s 2s/step - loss: 1.0873 - acc: 0.7566 - val_loss: 1.7073 - val_acc: 0.6174
Epoch 6/30
390/390 [==============================] - 931s 2s/step - loss: 0.9963 - acc: 0.7802 - val_loss: 1.4415 - val_acc: 0.6837
Epoch 7/30
390/390 [==============================] - 1045s 3s/step - loss: 0.9203 - acc: 0.7995 - val_loss: 1.7982 - val_acc: 0.6143
Epoch 8/30
390/390 [==============================] - 955s 2s/step -

## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠訓練!

最後一天的作業請閱讀這篇[非常詳盡的文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet

## 進階挑戰!
有志以成為機器學習工程師為目標的同學們可以參加這個挑戰，這[網站](https://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html#43494641522d3130)記錄了歷年來 Cifar-10 中排名最高的論文，請試著閱讀論文並撰寫出相對應的程式碼，復現出論文的結果。

這樣的能力在機器學習領域中是非常重要的，具備閱讀他人論文並實現的能力，可為自己在履歷上增添不少分數，當然難度也相當高，若是不透徹了解文章內容或是程式能力不夠紮實，可是不能復現別人辛苦的研究成果的喔! 就請各位同學好好努力，往自己的機器學習之路邁進吧:)

## Original testing result
Test loss: 1.3123285260200501
Test accuracy: 0.7493
## my final testing result
Test loss: 0.4895139602184296
Test accuracy: 0.9039
